In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.insert(0, '../osiris_query')
sys.path.insert(0, '../pandas_glit/')
import src.query as qry
import report.report as report
import pandas as pd
import datetime as dt
now = dt.datetime.now().strftime('%Y%m%d')
forms = 's_ooa_dos_BA1920_MATCHING_I'

In [6]:
df_wg = qry.load_frame('ad_hoc_sw_matching_werkgroepen')
opl = ['SOCB', 'CULB']
query = "IO_PROCES == 'BA1920_MATCHING' and OPLEIDING in @opl"
df_io = qry.load_frame('s_ooa_2019').query(query)
df_wg = df_wg.merge(df_io[['IO_AANVR_ID', 'STUDENTNUMMER', 'OPLEIDING', 'IO_PROCES']], how='left', on='STUDENTNUMMER')

wg_culb_1 = df_wg.query("CURSUS == '201200040' and GROEPSNUMMER == 1")['IO_AANVR_ID']
wg_culb_2 = df_wg.query("CURSUS == '201200040' and GROEPSNUMMER == 2")['IO_AANVR_ID']
wg_socb_1 = df_wg.query("CURSUS == '201200038' and GROEPSNUMMER == 1")['IO_AANVR_ID']
wg_socb_2 = df_wg.query("CURSUS == '201200038' and GROEPSNUMMER == 2")['IO_AANVR_ID']

dfs = {
    'wrkgrp_culb_1': wg_culb_1, 
    'wrkgrp_culb_2': wg_culb_2, 
    'wrkgrp_socb_1': wg_socb_1, 
    'wrkgrp_socb_2': wg_socb_2,
    }

for df in dfs:
    wrkgrp = dfs[df]
    df_forms = qry.load_frame(forms).query("IO_AANVR_ID in @wrkgrp")
    pack = qry.pack_data(df_forms, df, '', '')
    qry.save_datapack(df, pack)

FileNotFoundError: [Errno 2] No such file or directory: '\\\\soliscom.uu.nl\\uu\\Data\\UBD\\FCA\\ASC\\05. CSA\\70. Python\\osiris_query\\output\\ad_hoc_sw_matching_werkgroepen.pkl'

In [ ]:
matching_date = 'JUNI'
lang = 'nl'
for df in dfs:
    naam_elem = df.split('_')
    werkgroep = naam_elem[2]
    programme = naam_elem[1].upper()
    page = report.run(df, programme, matching_date, lang=lang)
    if page is not None:
        if not isinstance(matching_date, list):
            matching_date = [matching_date]
        filename = f"output/{now}_{programme}_werkgroep{werkgroep}_{'_'.join(matching_date)}_2019_stat_matchingformulier.html"
        with open(filename, 'w', encoding='utf-16') as f:
            f.write(page.html)